In [ ]:
!pip install lightning
!pip install pytorch-forecasting
!pip install examples

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')
import pickle
from tqdm import tqdm
import torch
import copy

# import pytorch_lightning as pl
# from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
# from pytorch_lightning.loggers import TensorBoardLogger
# from lightning.pytorch.tuner import Tuner
import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.tuner import Tuner

from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, Baseline
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss

# from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

# prompt: gpu 사용 가능한지 확인하는 코드

if torch.cuda.is_available():
  print("GPU is available")
  device = torch.device("cuda")
else:
  print("GPU is not available")
  device = torch.device("cpu")

GPU is not available


In [2]:
%cd drive/MyDrive/ime_project/
!ls

/content/drive/MyDrive/ime_project
capston_EDA.ipynb  EDA_1003.ipynb  EDA_1101.ipynb	     feature_selection.ipynb
data		   EDA_1010.ipynb  EDA_1102.ipynb	     result
DL		   EDA_1019.ipynb  EDA_1201.ipynb	     trial.ipynb
done		   EDA_1030.ipynb  feature_selection2.ipynb  var.ipynb


In [11]:
data = pd.read_csv("result/dataset.csv")#, index_col="Unnamed: 0")
data.rename(columns={'Unnamed: 0':'time_idx'},inplace=True)
data["time_idx"] = pd.to_datetime(data["time_idx"])
print(len(data.columns))
data.sample(10, random_state=777)

87


,time_idx,Ahu1_inv,Ahu2_inv,Ahu3_inv,Ahu4_inv,Ahu5_inv,Ahu6_inv,Ahu7_inv,Ahu8_inv,Ahu9_inv,...,Plant9_pm10,Plant9_temperature,Plant9_humidity,Plant10_air,Plant10_monoxide,Plant10_dioxide,Plant10_pm2d5,Plant10_pm10,Plant10_temperature,Plant10_humidity
24171,2023-06-18 04:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,29.08,63.08,72.0,4.0,532.0,10.0,11.0,29.35,61.86
10967,2023-03-18 11:50:00,40.0,40.0,40.0,40.0,40.0,35.0,40.0,40.0,40.0,...,29.0,31.42,48.77,123.0,2.0,439.0,27.0,30.0,31.88,47.56
7239,2023-02-20 14:30:00,32.0,32.0,32.0,0.0,31.0,0.0,31.0,31.0,31.0,...,20.0,32.47,33.19,72.0,0.0,417.0,19.0,21.0,32.94,32.27
28439,2023-07-17 19:50:00,40.0,35.0,40.0,40.0,40.0,35.0,40.0,40.0,40.0,...,1.0,31.00,46.96,84.0,4.0,506.0,1.0,1.0,31.21,45.87
5985,2023-02-11 21:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,28.40,59.22,18.0,0.0,439.0,10.0,11.0,28.42,58.98
22198,2023-06-04 11:40:00,45.0,35.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,...,1.0,34.21,42.67,59.0,2.0,469.0,3.0,3.0,34.26,42.57
14429,2023-04-11 12:50:00,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,...,30.0,35.20,44.87,547.0,0.0,478.0,38.0,43.0,35.17,44.88
7889,2023-02-25 02:50:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,37.0,28.91,32.48,0.0,1.0,430.0,39.0,44.0,29.39,31.48
5714,2023-02-10 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,28.13,62.12,0.0,0.0,451.0,6.0,6.0,28.20,61.89
26662,2023-07-05 11:40:00,45.0,35.0,45.0,45.0,45.0,35.0,45.0,45.0,45.0,...,12.0,30.84,51.35,198.0,4.0,552.0,13.0,14.0,31.16,50.55


In [12]:
data.columns

Index(['time_idx', 'Ahu1_inv', 'Ahu2_inv', 'Ahu3_inv', 'Ahu4_inv', 'Ahu5_inv',
       'Ahu6_inv', 'Ahu7_inv', 'Ahu8_inv', 'Ahu9_inv', 'Ahu10_inv',
       'Ahu11_inv', 'Ahu12_inv', 'Ahu13_inv', 'Ahu14_inv', 'Chiller1_inv',
       'Chiller2_inv', 'Plant1_air', 'Plant1_monoxide', 'Plant1_dioxide',
       'Plant1_pm2d5', 'Plant1_pm10', 'Plant1_temperature', 'Plant1_humidity',
       'Plant2_air', 'Plant2_monoxide', 'Plant2_dioxide', 'Plant2_pm2d5',
       'Plant2_pm10', 'Plant2_temperature', 'Plant2_humidity', 'Plant3_air',
       'Plant3_monoxide', 'Plant3_dioxide', 'Plant3_pm2d5', 'Plant3_pm10',
       'Plant3_temperature', 'Plant3_humidity', 'Plant4_air',
       'Plant4_monoxide', 'Plant4_dioxide', 'Plant4_pm2d5', 'Plant4_pm10',
       'Plant4_temperature', 'Plant4_humidity', 'Plant5_air',
       'Plant5_monoxide', 'Plant5_dioxide', 'Plant5_pm2d5', 'Plant5_pm10',
       'Plant5_temperature', 'Plant5_humidity', 'Plant6_air',
       'Plant6_monoxide', 'Plant6_dioxide', 'Plant6_pm2d5', 'Pl

In [25]:
max_prediction_length = 6*6
max_encoder_length = 6*6*4
training_cutoff = data["time_idx"].max() - pd.to_timedelta(max_prediction_length*10, unit='T')

In [ ]:
training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx", # time index가 무엇인가
    target="volume", # predict 하려는 예측 target => list
    group_ids=["agency", "sku"], # list of column names identifying a time series (해당 row의 key가 되는 feature)
    min_encoder_length=max_encoder_length // 2,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,

    static_categoricals=["agency", "sku"], # 시간이 지나도 변하지 않는 continuous feature => list
    static_reals=["avg_population_2017", "avg_yearly_household_income_2017"], # 시간이 지나도 변하지 않는 categorical feature => list

    time_varying_known_reals=["time_idx", "price_regular", "discount_in_percent"], # 시간이 지남에 따라 변경되고 미래에 알려지는 continuous feature => list
    time_varying_known_categoricals=["special_days", "month"], # 시간이 지남에 따라 변경되고 미래에 알려지는 categorical feature => list

    # variable_groups={"special_days": special_days}, # 데이터의 열 목록에 이름을 매핑하는 dictionary

    time_varying_unknown_categoricals=[], # 시간이 지남에 따라 변경되고 미래에 알려지지 않는 categorical feature => list
    time_varying_unknown_reals=[ # 시간이 지남에 따라 변경되고 미래에 알려지지 않는 continuous feature => list
        "volume",
        "log_volume",
        "industry_volume",
        "soda_volume",
        "avg_max_temp",
        "avg_volume_by_agency",
        "avg_volume_by_sku",
    ],

    target_normalizer=GroupNormalizer( # 대상을 정규화하기 위해 group_ids, target 및 time_idx를 사용하는 변환기
        groups=["agency", "sku"], transformation="softplus"
    ),  # use softplus and normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)